In [1]:
import numpy as np #numerical computation
import pandas as pd #data wrangling
import matplotlib.pyplot as plt #plotting package
#Next line helps with rendering plots
%matplotlib inline
import matplotlib as mpl #add'l plotting functionality
mpl.rcParams['figure.dpi'] = 400 #high res figures
#import graphviz #to visualize decision trees

In [3]:
df = pd.read_csv("C:/Users/kavya/desktop/transfusion_data.csv")

In [4]:
df.rename(columns = {'whether he/she donated blood in March 2007':'target'}, inplace=True)

In [5]:
df_zero_mask = df == 0

In [8]:
feature_zero_mask = df_zero_mask.iloc[:,:].all(axis=1)

In [9]:
#checking if there are any redundant data rows with invalid values
sum(feature_zero_mask)

0

In [10]:
df.head(2)

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),target
0,2,50,12500,98,1
1,0,13,3250,28,1


In [12]:
df.target.value_counts()

0    570
1    178
Name: target, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = \
train_test_split(df.drop('target', axis=1), df['target'].values,
test_size=0.2, random_state=24, stratify=df.target)

In [17]:
X_train.head()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
236,11,3,750,15
676,14,3,750,26
512,2,11,2750,28
172,2,1,250,2
359,21,16,4000,64


In [19]:
!pip install tpot
from tpot import TPOTClassifier

  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11959 sha256=639f5b36fede16d4618a41ffb952ca78fcd9755d8eb43775450bb02057e48e1f
  Stored in directory: c:\users\kavya\appdata\local\pip\cache\wheels\07\2e\ce\e558b7d4f9aafcdc0e5638ef890a3d5166d8a0f2c2dc768379
Successfully built stopit


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\users\kavya\appdata\local\programs\python\python36\python.exe -m pip install --upgrade pip' command.
c:\users\kavya\appdata\local\programs\python\python36\lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [20]:
pipeline_optimizer = TPOTClassifier()

In [21]:
pipeline_optimizer = TPOTClassifier(generations=5, population_size=20, cv=5,
                                    random_state=24, verbosity=2)

pipeline_optimizer.fit(X_train, y_train)


Generation 1 - Current best internal CV score: 0.7776890756302521
Generation 2 - Current best internal CV score: 0.7911344537815126
Generation 3 - Current best internal CV score: 0.7944397759103642
Generation 4 - Current best internal CV score: 0.7978291316526611
Generation 5 - Current best internal CV score: 0.7978291316526611
Best pipeline: MLPClassifier(RobustScaler(input_matrix), alpha=0.001, learning_rate_init=0.1)


TPOTClassifier(generations=5,
               log_file=<ipykernel.iostream.OutStream object at 0x000000CEAEB0CE80>,
               population_size=20, random_state=24, verbosity=2)

In [22]:
print(pipeline_optimizer.score(X_test, y_test))

0.8133333333333334


In [23]:
X_train.var()

Recency (months)         6.723867e+01
Frequency (times)        2.980751e+01
Monetary (c.c. blood)    1.862970e+06
Time (months)            5.751160e+02
dtype: float64

In [34]:
#correcting the variance using log normalization
X_train_normalized, X_test_normalized = X_train.copy(), X_test.copy()
for df_ in [X_train_normalized, X_test_normalized]:
   
    df_['Monetary_log'] = np.log(df_['Monetary (c.c. blood)'])
  
    df_.drop(columns='Monetary (c.c. blood)', inplace=True)

X_train_normalized.var()

Recency (months)      67.238668
Frequency (times)     29.807513
Time (months)        575.116006
Monetary_log           0.819058
dtype: float64

In [38]:
from sklearn.linear_model import LogisticRegression 

In [40]:
lg = LogisticRegression(random_state=24)
lg.fit(X_train_normalized, y_train)

LogisticRegression(random_state=24)

In [43]:
from sklearn.metrics import roc_auc_score
y_predict_proba = lg.predict_proba(X_test_normalized)
roc_auc_score(y_test, y_predict_proba[:,1])

0.7877680311890839

In [45]:
import pickle
with open('blood_donation.pickle','wb') as f:
    pickle.dump(lg,f)

In [46]:
final_model = pickle.load(open('blood_donation.pickle', 'rb'))

In [47]:
predicts = final_model.predict([[23,6,1500,45]])

print(round(predicts[0],2))

0


In [50]:
x = predicts = final_model.predict([[14,2,500,35]])
print(round(predicts[0],2))

1


In [51]:
print(round(x[0],2)*100)

100
